<a href="https://colab.research.google.com/github/ambitama-poddar/Machine-Learning-Practicals/blob/main/DA_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ITA6016 Machine Learning**

**Lab Assessment Task - 3**

**-Submitted by Ambitama Poddar(22MCA0106)**

# RNN model for analysing the sentiment of the tweets using “Sentiment140 dataset”

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
Path = 'drive/My Drive/Colab Notebooks'

In [25]:
df = pd.read_csv(Path+'/Sentiment140_dataset.csv',delimiter=',',encoding='latin-1')
df.head(20)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
5,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
6,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
7,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
8,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?
9,0,1467812416,Mon Apr 06 22:20:16 PDT 2009,NO_QUERY,erinx3leannexo,spring break in plain city... it's snowing


In [26]:
columns_to_use=df.columns
print(columns_to_use)

Index(['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY',
       '_TheSpecialOne_',
       '@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D'],
      dtype='object')


In [27]:
df.drop(['1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 3 columns):
 #   Column                                                                                                               Non-Null Count    Dtype 
---  ------                                                                                                               --------------    ----- 
 0   0                                                                                                                    1599999 non-null  int64 
 1   _TheSpecialOne_                                                                                                      1599999 non-null  object
 2   @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D  1599999 non-null  object
dtypes: int64(1), object(2)
memory usage: 36.6+ MB


In [28]:
df.head()

,0,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,scotthamilton,is upset that he can't update his Facebook by ...
1,0,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,ElleCTF,my whole body feels itchy and like its on fire
3,0,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,joy_wolf,@Kwesidei not the whole crew


In [29]:
# Create a dictionary to map current column names to new column names
new_column_names = {"0":'v1',
                    '_TheSpecialOne_': 'v2',
                    "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D": 'v3',
                   }

# Rename the columns
df.rename(columns=new_column_names, inplace=True)

In [30]:
new_columns=df.columns
print(new_columns)

Index(['v1', 'v2', 'v3'], dtype='object')


In [31]:
X = df.v3
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [32]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)
X_train = X_train.str.lower()
X_test = X_test.str.lower()

In [33]:
from keras.utils import pad_sequences

max_words = 5000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [34]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,100,input_length=max_len)(inputs)
    layer = LSTM(128)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(128)(layer)  # Additional Dense layer
    layer = Activation('relu')(layer)  # Changed activation function to 'relu'
    layer = Dropout(0.3)(layer)  # Adjusted dropout rate
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [35]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 150, 100)          500000    
                                                                 
 lstm_1 (LSTM)               (None, 128)               117248    
                                                                 
 FC1 (Dense)                 (None, 256)               33024     
                                                                 
 activation_3 (Activation)   (None, 256)               0         
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               3289

In [36]:
model.fit(sequences_matrix,Y_train,batch_size=1024,epochs=20,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/20
1063/1063 [==============================] - 72s 65ms/step - loss: 0.4467 - accuracy: 0.7920 - val_loss: 0.4158 - val_accuracy: 0.8078
Epoch 2/20
1063/1063 [==============================] - 68s 64ms/step - loss: 0.4103 - accuracy: 0.8137 - val_loss: 0.4167 - val_accuracy: 0.8104


In [37]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [38]:
accr = model.evaluate(test_sequences_matrix,Y_test)

7500/7500 [==============================] - 36s 5ms/step - loss: 0.4153 - accuracy: 0.8116


In [39]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.415
  Accuracy: 0.812
